**Mount to Blob Storage Source**

In [117]:
from notebookutils import mssparkutils

# Azure storage access info
source_blob_account_name = 'datasnowman' # replace with your blob name
source_blob_container_name = 'tiphackathon' # replace with your container name
source_blob_relative_path = 'catalog_sales' # replace with your relative folder path
source_linked_service_name = 'AzureBlobStorageTIP' # replace with your linked service name

source_blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(source_linked_service_name)
#spark.conf.set('fs.azure.sas.{source_blob_container_name}.{source_blob_account_name}.blob.core.windows.net', blob_sas_token)
#print(source_blob_sas_token)

In [118]:
# Allow SPARK to access from Blob remotely
source_wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (source_blob_container_name, source_blob_account_name, source_blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (source_blob_container_name, source_blob_account_name), source_blob_sas_token)
print('Remote blob path: ' + source_wasbs_path)

In [119]:
mssparkutils.fs.unmount("/mnt/blobsource")

In [120]:
mssparkutils.fs.mount( 
    #"wasbs://tiphackathon@datasnowman.blob.core.windows.net/catalog_sales/",
    f"{source_wasbs_path}", 
    "/mnt/blobsource",
    {"linkedService":f"{source_linked_service_name}"} 
)

In [121]:
files = mssparkutils.fs.ls("/")
for file in files:
    print(file.name, file.isDir, file.isFile, file.path, file.size)

In [93]:
#jobId = mssparkutils.env.getJobId() 
#f = open(f"/synfs/{jobId}/mnt/blobsource/myFile.txt", "a") 
#f.write("Hello world.") 
#f.close()

In [122]:
jobId = mssparkutils.env.getJobId()
#print(jobId)
source_files = mssparkutils.fs.ls(f"synfs:/{jobId}/mnt/blobsource/catalog_sales/")
for file in source_files:
    print(file.name, file.isDir, file.isFile, file.path, file.size)

**Mount ADLS Storage Sink**

In [123]:
from notebookutils import mssparkutils

# Primary storage info
sink_account_name = 'synapsetiphackathon' # fill in your primary account name
sink_container_name = 'landingzone' # fill in your container name
sink_relative_path = 'raw' # fill in your relative folder path
sink_linked_service_name = 'AzureDataLakeStorageTIP' # replace with your linked service name

sink_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (sink_container_name, sink_account_name, sink_relative_path)
print('Primary storage account path: ' + sink_adls_path)

In [124]:
mssparkutils.fs.unmount("/mnt/adlssink")

In [125]:
mssparkutils.fs.mount( 
    # "abfss://tiphackathon@synapsetiphackathon.dfs.core.windows.net/",
    f"{sink_adls_path}", 
    "/mnt/adlssink", 
    {"linkedService":f"{sink_linked_service_name}"}
)

In [126]:
files = mssparkutils.fs.ls("/")
for file in files:
    print(file.name, file.isDir, file.isFile, file.path, file.size)

In [127]:
jobId = mssparkutils.env.getJobId()
#print(jobId)
sink_files = mssparkutils.fs.ls(f"synfs:/{jobId}/mnt/adlssink/raw")
for file in sink_files:
    print(file.name, file.isDir, file.isFile, file.path, file.size)

In [52]:
# mssparkutils.fs.cp -R f"synfs:/{jobId}/mnt/blobsource/catalog_sales/" f"synfs:/{jobId}/mnt/adlssink/raw"

mssparkutils.fs.cp(f'synfs:/{jobId}/mnt/blobsource/catalog_sales/', f'synfs:/{jobId}/mnt/adlssink/raw', True)# Set the third parameter as True to copy all files and directories recursively
mssparkutils.fs.cp(f'synfs:/{jobId}/mnt/blobsource/catalog_returns/', f'synfs:/{jobId}/mnt/adlssink/raw', True)# Set the third parameter as True to copy all files and directories recursively
mssparkutils.fs.cp(f'synfs:/{jobId}/mnt/blobsource/catalog_page/', f'synfs:/{jobId}/mnt/adlssink/raw', True)# Set the third parameter as True to copy all files and directories recursively
mssparkutils.fs.cp(f'synfs:/{jobId}/mnt/blobsource/warehouse/', f'synfs:/{jobId}/mnt/adlssink/raw', True)# Set the third parameter as True to copy all files and directories recursively